In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import numpy as np

In [85]:
import numpy as np
import utils
import torch 
from torch.autograd import Variable
from Modules import VAE
import torch.nn as nn
import torch.nn.functional as F

filename = "/home/wafa/Desktop/study/sketchRNN/aaron_sheep.npz"
load_data = np.load(filename, encoding = 'latin1')
train_set = load_data['train'][:200]
test_set = load_data['test'][:100]

nb_steps = 10
feature_len=5
batch_size = 6
max_seq_len=250

train_set = utils.DataLoader(train_set, batch_size)
test_set = utils.DataLoader(test_set, batch_size)

total images <= max_seq_len is 200
total images <= max_seq_len is 100


In [488]:
class Encoder(nn.Module):
    
    def __init__(self, strokeSize, batchSize, Nh, Nz):
        super(Encoder, self).__init__()
        self.Nz = Nz
        self.cell = nn.LSTM(strokeSize, Nh//2, 1, bidirectional=True, batch_first=True)
        self.mu = nn.Linear(Nh, Nz)
        self.sigma = nn.Linear(Nh, Nz)
        self.h0 = nn.Linear(Nz, Nh)

    def forward(self, x):
        _, (hn, cn) = self.cell(x)
        hn = Variable(torch.cat((hn.data[0],hn.data[1]),1))
        sigma = self.sigma(hn)
        mu = self.mu(hn)
        sigma = torch.exp( sigma * 0.5)
        eps = Variable(torch.randn(self.Nz))
        z = mu + sigma * eps
        return F.tanh(self.h0(z)), z

In [489]:
class Decoder(nn.Module):
    
    def __init__(self, strokeSize, batchSize, Nh, Nz, Ny):
        super(Decoder, self).__init__()
        self.Nh = Nh
        self.cell = nn.LSTM(strokeSize+Nz, Nh, 1, batch_first=True)
        self.y = nn.Linear(Nh, Ny)
    
    def forward(self, x):
        output, (hn, cn) = self.cell(x)
        #print(output.size())
        #print(output.contiguous().view(-1, self.Nh).size())
        output = output.contiguous().view(-1, self.Nh)
        y = self.y(output)
        return y  

In [490]:
class SketchRNN(nn.Module):
    
    def __init__(self, strokeSize, batchSize, Nh, Nz, Ny, max_seq_len):
        super(SketchRNN, self).__init__()
        self.batchSize = batchSize
        self.max_seq_len = max_seq_len
        self.Nz = Nz
        self.encoder = Encoder(strokeSize, batchSize, Nh, Nz)
        self.decoder = Decoder(strokeSize, batchSize, Nh, Nz, Ny)
        
    def forward(self, x):
        # we don't take the inisiale S0 in the encoder so we do the 1:
        h0, z = self.encoder(x[:, 1:250+1, :])
        #here we take S0
        new_input = torch.cat((x[:, :250, :], z.view(self.batchSize, 1, self.Nz).expand(self.batchSize, self.max_seq_len, self.Nz)), 2)
        y = self.decoder(new_input)
        return y


In [491]:
M = 5
sketchRnn = SketchRNN(5, 6, 100, 32, 6*M+3, 250)
_, x, s = train_set.random_batch()

x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
y = sketchRnn(x)
print(y.size()) # reshaped

# note that we don't have to follow the same order of y in the paper! here we take the first M elements as z_pi, the second M elements as z_mu1 ...
# we just have to respect this order in the loss computation, it makes it easier to user only pytorch and avoid numpy
z_pen_logits = y[:, -3:]
z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr = torch.chunk(y[:, :-3], 6, dim=1)
z_pi = F.sigmoid(z_pi)
z_pen_logits = F.sigmoid(z_pen_logits)
z_sigma1 = torch.exp(z_sigma1)
z_sigma2 = torch.exp(z_sigma2)
z_corr = F.tanh(z_corr)

torch.Size([1500, 33])


In [492]:
def bi_normal(x1, x2, mu1, mu2, s1, s2, rho):
    
    norm1 = x1 - mu1
    norm2 = x2 - mu2
    z = torch.div(norm1, s1).pow(2) + torch.div(norm2, s2).pow(2) - 2 * torch.div(torch.mul(rho, torch.mul(norm1, norm2)), torch.mul(s1,s2))
    coef = torch.exp(-z/(2*(1-rho.pow(2))))
    denom = 2 * F.math.pi * s1 * s2 * torch.sqrt(1-rho.pow(2))
    result = torch.div(coef, denom)
    return result
    

In [503]:
def loss(z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_pen_logits, x1, x2, pen):
    
    #TODO optimize the loop
    ls = bi_normal(x1, x2, z_mu1[:, 0], z_mu2[:, 0], z_sigma1[:, 0], z_sigma2[:, 0], z_corr[:, 0]).view(1500,1)
    for i in range(1, M):
        ls2 = bi_normal(x1, x2, z_mu1[:, i], z_mu2[:, i], z_sigma1[:, i], z_sigma2[:, i], z_corr[:, i]).view(1500,1)
        ls = torch.cat((ls,ls2), 1)
    ls = torch.mul(ls, z_pi)
    ls = torch.sum(ls, 1, keepdim=True) # TODO change to Ns not Nmax
    ls = - torch.log(ls + 1e-6)
    
    lp = torch.log(z_pen_logits)
    lp = torch.mul(lp, pen)
    lp = - torch.sum(lp, 1, keepdim=True)
    
    return ls+lp

In [506]:
x1 = target[:,0]
x2 = target[:,1]
pen = target[:,2:]
lr = loss(z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_pen_logits, x1, x2, pen)
lr = torch.mean(lr)
lr

Variable containing:
 7.7472
[torch.FloatTensor of size 1]

In [98]:
#OLD
_, x, s = train_set.random_batch()
x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
_, (h, c) = encoder(x)
h = torch.cat((h.data[0],h.data[1]),1)
h0, z = vae(Variable(h))

In [124]:
torch.cat((x, z.view(6, 1, 128).expand(6, 251, 128)), 2)
torch.cat((x, z.view(batch_size, 1, N_z).expand(batch_size, max_seq_len+1, N_z)), 2)

torch.Size([6, 251, 133])

In [95]:
class VAE(nn.Module):
    
    def __init__(self, inputSize, outputSize):
        super(VAE, self).__init__()
        self.outputSize = outputSize
        self.mu = nn.Linear(inputSize, outputSize)
        self.sigma = nn.Linear(inputSize, outputSize)
        self.h0 = nn.Linear(outputSize, inputSize)

    def forward(self, x):
        sigma = self.sigma(x)
        mu = self.mu(x)
        sigma = torch.exp( sigma * 0.5)
        eps = torch.randn(self.outputSize)
        eps = Variable(eps)
        z = mu + sigma * eps
        return F.tanh( self.h0(z)), z

In [ ]:
data = np.load('/home/wafa/Desktop/study/sketchRNN/aaron_sheep/train.npy' ,encoding = 'latin1')
def pad_batch(batch):
    """Pad the batch to be stroke-5 bigger format as described in paper."""
    result = np.zeros((6, 251, 5), dtype=float)
    for i in range(6):
        l = len(batch[i])
        result[i, 0:l, 0:2] = batch[i][:, 0:2]
        result[i, 0:l, 3] = batch[i][:, 2]
        result[i, 0:l, 2] = 1 - result[i, 0:l, 3]
        result[i, l:, 4] = 1
        # put in the first token, as described in sketch-rnn methodology
        result[i, 1:, :] = result[i, :-1, :]
        result[i, 0, :] = 0
        result[i, 0, 2] = 1 # setting S_0 from paper.
        result[i, 0, 3] = 0
        result[i, 0, 4] = 0
    return result

rnn = nn.LSTM(5, 251, 1)
nb=0
while nb+6 < 101:
    input = Variable(torch.from_numpy(pad_batch(data[nb:nb+6])).type(torch.FloatTensor))
    output, hn = rnn(input) 
    print(output.size())
    nb += 6